In [1]:
import sys
!{sys.executable} -m pip install ./fhnw_ds_hs2019_weatherstation_api/dist/fhnw_ds_hs2019_weatherstation_api-0.1-py3-none-any.whl

Processing ./fhnw_ds_hs2019_weatherstation_api/dist/fhnw_ds_hs2019_weatherstation_api-0.1-py3-none-any.whl


In [3]:
from fhnw_ds_hs2019_weatherstation_api import data_import as weather
import os

# DB and CSV config
config = weather.Config()
# define CSV path
config.historic_data_folder='.'+os.sep+'data'
# define DB host
config.db_host='localhost'

# connect to DB
weather.connect_db(config)
# clean DB
weather.clean_db(config)
# import historic data
weather.import_historic_data(config)
# import latest data (delta between last data point in DB and current time)
weather.import_latest_data(config, append_to_csv=False, periodic_read=False)

Load historic data for mythenquai ...
	Load ./data/messwerte_mythenquai_2007-2018.csv
	Load ./data/messwerte_mythenquai_2019.csv
Historic data for mythenquai loaded.
Load historic data for tiefenbrunnen ...
	Load ./data/messwerte_tiefenbrunnen_2007-2018.csv
	Load ./data/messwerte_tiefenbrunnen_2019.csv
Historic data for tiefenbrunnen loaded.
Query mythenquai at 2019-06-17
Handle mythenquai from 2019-06-17 14:00:00 to 2019-06-18 00:00:00
Query tiefenbrunnen at 2019-06-17
Handle tiefenbrunnen from 2019-06-17 14:00:00 to 2019-06-18 00:00:00
Query mythenquai at 2019-06-18
Handle mythenquai from 2019-06-18 00:10:00 to 2019-06-19 00:00:00
Query tiefenbrunnen at 2019-06-18
Handle tiefenbrunnen from 2019-06-18 00:10:00 to 2019-06-19 00:00:00
Query mythenquai at 2019-06-19
Handle mythenquai from 2019-06-19 00:10:00 to 2019-06-20 00:00:00
Query tiefenbrunnen at 2019-06-19
Handle tiefenbrunnen from 2019-06-19 00:10:00 to 2019-06-20 00:00:00
Query mythenquai at 2019-06-20
Handle mythenquai from 20